In [2]:
import pandas as pd
import numpy as np
import pickle
import csv

from progressbar import ProgressBar
import time

In [1]:
#import artm

In [4]:
#Обрабатываем данные из data1
data1 = pd.read_csv("dialogs_1.csv", sep = ",")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# %matplotlib inline
# data1.value_counts().plot(kind='bar')

In [88]:
#Первичная предобработка
#NaN = -1
not_columns = ['ID', 'Дата.изменения.статуса', 'Подписчики', 'Контент']
#Заменяем NaN на -1 для удобства
for column in data1.columns:
    if column not in not_columns:
        data1[str(column)] = data1[str(column)].apply(str)
data1 = data1.fillna({'ID': -1, 'Подписчики': -1})
data1['ID'] = data1['ID'].apply(int)
data1['Подписчики'] = data1['Подписчики'].apply(int)
data1['Контент'] = data1['Контент'].apply(str)
data1 = data1.drop('Тип.обращения', axis=1)
data1 = data1.drop('Дата.изменения.статуса', axis=1)
#Отсортировали по ID
data1 = data1.sort_values(by='ID', ascending=0)
#Пренебрегаем комментариями, не являющимися цельной перепиской
data1 = data1[data1['ID'] != -1].reset_index(drop=True)
data1 = data1.reset_index(drop=True)

In [95]:
# для данных тех. поддержки следует только поставить if data1['Тип'][row] == '1-ответ'
def prepare_data1(data1):
    
    ##Создаем датафрейм пользовательских текстовых диалогов для data1
    #Выделяем тексты пользовательских обращений для последующей кластеризации
    
    #Запоминаем номер первого инцидента
    start = data1['ID'][0]
    data1_clienttexts = pd.DataFrame(columns=['ID', 'Автор', 'Категория.автора', 'Контент', 'Хост','Подписчики', 'Тема', 'Локация', 'Тональность'])
    data1_clienttexts['Контент']= ['' for _ in range(len(data1))]
    data1_clienttexts = data1_clienttexts.fillna('')
    #index - номер строки нового датафрейма
    index=0
    #first - первый ли текст в конкретной переписке
    first = True
    for row in range(len(data1)):
        #Проверка на -1 тут бесполезна, удалили ранее комментарии
        if data1['ID'][row] != -1:
            #если встретили инцидент впервые
            if data1['ID'][row] == start:
                if data1['Тип'][row] != '1-ответ':
                    data1_clienttexts.iloc[index]['Контент'] += ' ' + data1.iloc[row]['Контент']
        
            else:
                #Новый инцидент
                start = data1['ID'][row]
                if data1['Тип'][row] != '1-ответ':
                    first = True
                    index += 1
                    data1_clienttexts.iloc[index]['Контент'] += ' ' + data1.iloc[row]['Контент']
        
            if first:
                #Записываем все остальные поля, кроме текста
                if data1['Тип'][row] != '1-ответ':
                    first = False
                    data1_clienttexts.iloc[index]['ID'] = data1.iloc[row]['ID']
                    data1_clienttexts.iloc[index]['Автор'] = data1.iloc[row]['Автор']
                    data1_clienttexts.iloc[index]['Категория.автора'] = data1.iloc[row]['Категория.автора']
                    data1_clienttexts.iloc[index]['Хост'] = data1.iloc[row]['Хост']
                    data1_clienttexts.iloc[index]['Подписчики'] = data1.iloc[row]['Подписчики']
                    data1_clienttexts.iloc[index]['Тема'] = data1.iloc[row]['Тема']
                    data1_clienttexts.iloc[index]['Локация'] = data1.iloc[row]['Локация']
                    try:
                        data1_clienttexts.iloc[index]['Тональность'] = data1[data1['ID'] == data1.iloc[row]['ID']]['Тональность'].unique()[1]
                    except IndexError:
                        data1_clienttexts.iloc[index]['Тональность'] = data1[data1['ID'] == data1.iloc[row]['ID']]['Тональность'].unique()[0]
        pbar.update(row)
        
    data1_clienttexts['Контент'].replace('', np.nan, inplace=True)
    data1_clienttexts = data1_clienttexts.dropna() 
    
    
    return data1_clienttexts

In [96]:
pbar = ProgressBar(maxval=len(data1))
pbar.start()
data1_clienttexts = prepare_data1(data1)
pbar.finish()

100% |########################################################################|


In [3]:
#data1_clienttexts['Подписчики'].value_counts()

In [97]:
data1_clienttexts.to_pickle('data1_clienttexts.pickle')

In [5]:
#len(data1_clienttexts)

In [4]:
#data1_clienttexts